<a href="https://colab.research.google.com/github/AnaNicoleMassaneiro/AnaNicoleMassaneiro/blob/main/Copie_de_segmentacao_separacao_cores_de_celulas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ki-67 - Segmentação e Contagem de Células

*   Item da lista
*   Item da lista



In [1]:
# Libs
import os
import glob
import math
import cv2
import numpy as np
import matplotlib.pyplot as plt
import zipfile
from sklearn.metrics import mean_squared_error
import pandas as pd
from ipywidgets import interact, widgets
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np


* Para acessar as imagens é necessário baixá-las e adicioná-las ao Jupyter Notebook

> Bloco com recuo


* As pastas de cada lâmina podem ser baixadas pelo drive no formato .zip

In [2]:
# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Definir o caminho completo para os arquivos zip dentro da pasta "validacao"
zip_file1_path = '/content/drive/My Drive/validacao.zip'

# Descompactar os arquivos
!unzip -o "$zip_file1_path" -d /content/

Mounted at /content/drive
Archive:  /content/drive/My Drive/validacao.zip
  inflating: /content/Validação/20_168_82.tiff  
  inflating: /content/Validação/20_346_31.tiff  
  inflating: /content/Validação/20_057_77.tiff  
  inflating: /content/Validação/20_057_61.tiff  
  inflating: /content/Validação/20_228_46.tiff  
  inflating: /content/Validação/20_346_34.tiff  
  inflating: /content/Validação/20_346_23.tiff  
  inflating: /content/Validação/20_156_36.tiff  
  inflating: /content/Validação/20_346_38.tiff  
  inflating: /content/Validação/20_057_64.tiff  
  inflating: /content/Validação/20_156_91.tiff  
  inflating: /content/Validação/20_228_54.tiff  
  inflating: /content/Validação/20_201_72.tiff  
  inflating: /content/Validação/20_228_58.tiff  
  inflating: /content/Validação/20_228_55.tiff  
  inflating: /content/Validação/20_057_79.tiff  
  inflating: /content/Validação/20_201_54.tiff  
  inflating: /content/Validação/20_156_83.tiff  
  inflating: /content/Validação/20_236_48.ti

## Funções
* Normalização Reinhard

In [3]:
### REINHARD NORMALIZATION
### https://github.com/DigitalSlideArchive/HistomicsTK/blob/master/histomicstk/preprocessing/color_normalization/reinhard.py

# ----------------------------------------------------------------------------------------------------------------------
# Define conversion matrices
## RGB to LAB
_rgb2lms = np.array([[0.3811, 0.5783, 0.0402],
                     [0.1967, 0.7244, 0.0782],
                     [0.0241, 0.1288, 0.8444]])

_lms2lab = np.dot(
    np.array([[1 / (3**0.5), 0, 0],
              [0, 1 / (6**0.5), 0],
              [0, 0, 1 / (2**0.5)]]),
    np.array([[1, 1, 1],
              [1, 1, -2],
              [1, -1, 0]])
)
# ----------------------------------------------------------------------------------------------------------------------

# Define conversion matrices
## LAB to RGB
_lms2rgb = np.linalg.inv(_rgb2lms)
_lab2lms = np.linalg.inv(_lms2lab)
# ----------------------------------------------------------------------------------------------------------------------

# Function that converts an RGB image to LAB
def rgb_to_lab(im_rgb):

    # Get input image dimensions
    m = im_rgb.shape[0]
    n = im_rgb.shape[1]

    # Calculate im_lms values from RGB
    im_rgb = np.reshape(im_rgb, (m * n, 3))
    im_lms = np.dot(_rgb2lms, np.transpose(im_rgb))
    im_lms[im_lms == 0] = np.spacing(1)

    # Calculate LAB values from im_lms
    im_lab = np.dot(_lms2lab, np.log(im_lms))

    # Reshape to 3-channel image
    im_lab = np.reshape(im_lab.transpose(), (m, n, 3))

    return(im_lab)
# ----------------------------------------------------------------------------------------------------------------------

# Function that converts an LAB image to RGB
def lab_to_rgb(im_lab):

    # Get input image dimensions
    m = im_lab.shape[0]
    n = im_lab.shape[1]

    # Calculate im_lms values from LAB
    im_lab = np.reshape(im_lab, (m * n, 3))
    im_lms = np.dot(_lab2lms, np.transpose(im_lab))

    # Calculate RGB values from im_lms
    im_lms = np.exp(im_lms)
    im_lms[im_lms == np.spacing(1)] = 0

    im_rgb = np.dot(_lms2rgb, im_lms)

    # Reshape to 3-channel image
    im_rgb = np.reshape(im_rgb.transpose(), (m, n, 3))

    return(im_rgb)
# ----------------------------------------------------------------------------------------------------------------------

# Function that applies color normalization (Reinhard Normalization)
def reinhard(im_src, target_mu, target_sigma, src_mu=None, src_sigma=None, mask_out=None):

    # convert input image to LAB color space
    im_lab = rgb_to_lab(im_src)

    # mask out irrelevant tissue / whitespace / etc
    if mask_out is not None:
        mask_out = mask_out[..., None]
        im_lab = np.ma.masked_array(
            im_lab, mask=np.tile(mask_out, (1, 1, 3)))

    # calculate src_mu and src_sigma if either is not provided
    if (src_mu is None) or (src_sigma is None):
        src_mu = [im_lab[..., i].mean() for i in range(3)]
        src_sigma = [im_lab[..., i].std() for i in range(3)]

    # scale to unit variance
    for i in range(3):
        im_lab[:, :, i] = (im_lab[:, :, i] - src_mu[i]) / src_sigma[i]

    # rescale and recenter to match target statistics
    for i in range(3):
        im_lab[:, :, i] = im_lab[:, :, i] * target_sigma[i] + target_mu[i]

    # convert back to RGB colorspace
    im_normalized = lab_to_rgb(im_lab)
    im_normalized[im_normalized > 255] = 255
    im_normalized[im_normalized < 0] = 0

    # return masked values and reconstruct unmasked LAB image
    if mask_out is not None:
        im_normalized = im_normalized.data
        for i in range(3):
            original = im_src[:, :, i].copy()
            new = im_normalized[:, :, i].copy()
            original[np.not_equal(mask_out[:, :, 0], True)] = 0
            new[mask_out[:, :, 0]] = 0
            im_normalized[:, :, i] = new + original
    im_normalized = im_normalized.astype(np.uint8)

    return(im_normalized)
# ----------------------------------------------------------------------------------------------------------------------

* Leitura das imagens
* Extração das regiões marrons e azuis
* Gráficos de comparação

In [4]:
# ----------------------------------------------------------------------------------------------------------------------

# Function that read the images and store the relevant information
# Function to read images and store relevant information
def read_images(scan_dir, target_image_path, norm=True):
    data = {
        "images": [],
        "scores": [],
        "filenames": [],
    }

    # Load the target image
    target_image = cv2.imread(target_image_path)
    target_image = cv2.cvtColor(target_image, cv2.COLOR_BGR2RGB)
    target_lab = rgb_to_lab(target_image)
    l, a, b = cv2.split(target_lab)
    target_mean = np.array([np.mean(l), np.mean(a), np.mean(b)])
    target_std = np.array([np.std(l), np.std(a), np.std(b)])

    # Get the paths to all images inside the given folder
    image_paths = glob.glob("{}/*".format(scan_dir))

    for path in image_paths:
        # Read image
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Apply Reinhard Normalization
        if norm:
            image = reinhard(image, target_mean, target_std)

        # Get the score
        score = int(path.split("/")[-1].split("_")[-1][0:2])

        # Store the information
        data["images"].append(image)
        data["scores"].append(score)
        data["filenames"].append(path)

    return data

# Function that enhances the image contrast with the CLAHE technique
def enhance_contrast(rgb_image):

  # Transform it to LAB Color Space
  lab = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2LAB)

  # Split the channels --> L [lightness] | A [red/green] | B [blue/yellow])
  l, a, b = cv2.split(lab)

  # CLAHE - Contrast Limited Adaptive Histogram Equalization
  ## Split the image in tiles (8x8) and apply a histogram equalization considering a limit to the contrast (2.0)
  ## Ideia: some different objects have different colors and lightness
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

  # The CLAHE will be applied on the L channel (lightness)
  clahe_l = clahe.apply(l)

  # Combine the enhanced L channel with A and B channels
  enhanced_lab = cv2.merge((clahe_l, a, b))

  # Convert the image from LAB to RGB
  enhanced_img = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2RGB)

  return (enhanced_img)
# ----------------------------------------------------------------------------------------------------------------------

# Function that generates a mask with the regions that contain the informed color
def color_extraction(image, color, exp=4, kernel_size=2, limits=None):

  # Convert to HSV color space
  hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

  # Define the limits
  if(limits is None):
    limits = {
        'brown1': [[0, 20, 20], [25, 250, 195]],
        'brown2': [[170, 20, 20], [180, 250, 195]],
        'blue': [[90, 20, 20], [140, 250, 240]],
        'black': [[0, 0, 0], [200, 50, 80]]
}

  # Image shape (rows, cols)
  shape = np.shape(image)

  # Image type
  type_ = type(image[0,0,0])

  # Create a black (all zeros) mask
  final_mask = np.zeros((shape[0], shape[1]), dtype=type_)

  masks = []
  for c in color:
    # Set color limits
    lower_limit = np.array([limits[c][0]])
    upper_limit = np.array([limits[c][1]])

    # Create mask
    masks.append(cv2.inRange(hsv_image, lower_limit, upper_limit))

  # Combine the masks
  for m in masks:
    final_mask += m

  if(exp != 1):

    # Apply matematical morphology to supress the noise (opening)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_size, kernel_size))
    final_mask = cv2.erode(final_mask, kernel)
    final_mask = cv2.dilate(final_mask, kernel)

  # Fill contours that touch the image border
  contours, _ = cv2.findContours(final_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  if(exp == 3 or exp == 4):
    for contour in contours:
      x, y, w, h = cv2.boundingRect(contour)
      if x == 0 or y == 0 or x + w == final_mask.shape[1] or y + h == final_mask.shape[0]:
        cv2.drawContours(final_mask, [contour], -1, 0, -1)

  # Change the 255 value to 1
  final_mask[final_mask == 255] = 1

  # Transform the image of 1 channel to 3
  mask = cv2.merge([final_mask, final_mask, final_mask])

  # Get the segmentation
  segmentation = cv2.multiply(image, mask.astype(np.uint8))

  return(segmentation, final_mask)
# ----------------------------------------------------------------------------------------------------------------------

# Function that plots 3 images side by side
def comparison_plot(original_image, seg_o, seg_e, title="", images_title=("", "", "")):

  # Plot config
  subplot_config = (1, 3)
  plt.rcParams['figure.figsize'] = [17, 7]
  figure, axes = plt.subplots(subplot_config[0], subplot_config[1])
  figure.subplots_adjust(top=0.95)

  # Adjust the info for each subplot
  ## Original image
  axes[0].imshow(original_image, cmap="gray")
  axes[0].axis('off')
  axes[0].set_title(images_title[0])

  # Segmentation without contrast enhancement
  axes[1].imshow(seg_o, cmap="gray")
  axes[1].axis('off')
  axes[1].set_title(images_title[1])

  # Segmentation with contrast enhancement
  axes[2].imshow(seg_e, cmap="gray")
  axes[2].axis('off')
  axes[2].set_title(images_title[2])

  plt.suptitle(title)
  plt.show()
# ----------------------------------------------------------------------------------------------------------------------

# Calculates the Ki-67 score for the given image
def get_image_ki67_score(original_image, exp=4, show=False):

  # Apply blur (remove noise)
  image = cv2.GaussianBlur(original_image, (5, 5), 0)

  # Brown cells
  _, brown_mask = color_extraction(image, ['brown1', 'brown2'], exp=exp)

  # Blue cells
  _, blue_mask = color_extraction(image, ['blue'], exp=exp)

  # Black stains
  _, black_mask = color_extraction(image, ['black'], exp=exp)

  # Increase the area of the black stain (get the border)
  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(25, 25))
  black_mask = cv2.dilate(black_mask, kernel)

  # Ignore the black stain (blue segmentation)
  blue_mask[black_mask == 1] = 0

  # Get the brown contours of the image (removing inner contours)
  positive_contours, _ = cv2.findContours(brown_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Filter the contours with area > 100 (adjust this threshold as needed)
  positive_contours = [contour for contour in positive_contours if cv2.contourArea(contour) > 100]

  # Get the blue contours of the image (removing inner contours)
  negative_contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

  # Filter the contours with area < 50
  negative_contours = [contour for contour in negative_contours if(cv2.contourArea(contour) > 45)]

  # Get the area of each positive contour
  positive_areas = [cv2.contourArea(contour) for contour in positive_contours]

  # Get the area of each negative contour
  negative_areas = [cv2.contourArea(contour) for contour in negative_contours]

  # Calc the score
  total_area = sum(positive_areas) + sum(negative_areas)

  if total_area > 0:
      positive_area_ratio = sum(positive_areas) / total_area
      positive_area_ratio = float(round(positive_area_ratio * 100.0))
  else:
      positive_area_ratio = 0


  if(show):

    # Update the segmented image
    channel_brown = cv2.drawContours(np.zeros(original_image.shape), positive_contours, -1, (1, 1, 1), -1).astype(np.uint8)
    channel_blue = cv2.drawContours(np.zeros(original_image.shape), negative_contours, -1, (1, 1, 1), -1).astype(np.uint8)

    # Get the segmentation
    brown_seg = cv2.multiply(original_image, channel_brown)
    blue_seg = cv2.multiply(original_image, channel_blue)

    # Plot the results
    comparison_plot(original_image, brown_seg, blue_seg,
                  title="Extração de Cor", images_title=("Imagem Original", "Células Positivas", "Células Negativas"))

  return(positive_area_ratio)
# ----------------------------------------------------------------------------------------------------------------------

# Calculates Ki-67 scores for all images inside the given folder
def get_dir_ki67_scores(scan_dir, norm=True, exp=4, show=False):
    print("{}:".format(scan_dir))

    if(exp == 4):
      norm = True

    # Read the info
    data = read_images(scan_dir, norm)

    # Add the results into the dictionary
    data["results"] = []

    # Create a list to store MSE for each image
    rmse_list = []

    # For each image
    for image, score, path in zip(data["images"], data["scores"], data["filenames"]):

        # Get the Ki-67 score
        positive_area_ratio = get_image_ki67_score(image, exp=exp, show=show)

        print("\nImagem: {}".format(path.split("/")[-1]))
        print(" - Ki-67 Esperado: {}".format(score))
        print(" - Ki-67 Obtido: {}".format(int(positive_area_ratio)))

        # Store the result
        data["results"].append(positive_area_ratio)

        # Calculate the Root Mean Squared Error (MSE) for this image
        mse = mean_squared_error([float(score)], [positive_area_ratio])
        rmse = math.sqrt(mse)
        rmse_list.append(rmse)

        print(" - Raíz Quadrada do Erro Quadrático Médio (RMSE): {}".format(rmse))

    # Calculate the average and std RMSE of all images
    average_rmse = np.mean(rmse_list)
    std_rmse = np.std(rmse_list)

    # Calculate the average obtained Ki-67 score for all images
    average_obtained_ki67_score = np.mean(data["results"])

    print("\nResultados:")
    print(" - RMSE Médio: {:.2f}".format(average_rmse))
    print(" - RMSE Desvio Padrão: {:.2f}".format(std_rmse))
    print(" - Ki-67 Médio Obtido: {:.2f}".format(average_obtained_ki67_score))

    return(data)
# ----------------------------------------------------------------------------------------------------------------------

## Sliders
* O código abaixo apresenta uma forma semi-automática para ajustar os parâmetros das imagens.
* Podem ser controlados os brilhos considerados de cada cor para realizar a segmentação.
* O **brown 1** é referente a tonalidade mais escura de marrom, sendo seu valor padrão de 195.
* O **brown 2** é referente a tonalidade mais clara de marrom, sendo seu valor padrão de 195.
* O **blue** é referente a tonalidade azul, sendo seu valor padrão 240.

In [5]:
# Install JupyterLite
!pip install -q ipywidgets
# ----------------------------------------------------------------------------------------------------------------------

# Import all interactions
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
# ----------------------------------------------------------------------------------------------------------------------

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.5 MB/s eta 0:00:00


* Função que realiza a segmentação.
* Ela foi adaptada para que seja possível avaliar diferentes valores no gráfico de interação.

In [6]:
# ----------------------------------------------------------------------------------------------------------------------

# Calculates the Ki-67 score for the given image
def evaluate_color_separation(original_image, score, kernel_size=2, exp=4, limits=None):

  # Apply blur (remove noise)
  image = cv2.GaussianBlur(original_image, (5, 5), 0)

  # Brown cells
  _, brown_mask = color_extraction(image, ['brown1', 'brown2'], exp=exp, kernel_size=kernel_size, limits=limits)

  # Blue cells
  _, blue_mask = color_extraction(image, ['blue'], exp=exp, kernel_size=kernel_size, limits=limits)

  # Black stains
  _, black_mask = color_extraction(image, ['black'], exp=exp)

  # Increase the area of the black stain (get the border)
  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(25, 25))
  black_mask = cv2.dilate(black_mask, kernel)

  # Ignore the black stain (blue segmentation)
  blue_mask[black_mask == 1] = 0

  # Get the brown contours of the image (removing inner contours)
  positive_contours, _ = cv2.findContours(brown_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Filter the contours with area > 100 (adjust this threshold as needed)
  positive_contours = [contour for contour in positive_contours if cv2.contourArea(contour) > 100]

  # Get the blue contours of the image (removing inner contours)
  negative_contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

  # Filter the contours with area < 50
  negative_contours = [contour for contour in negative_contours if(cv2.contourArea(contour) > 45)]

  # Get the area of each positive contour
  positive_areas = [cv2.contourArea(contour) for contour in positive_contours]

  # Get the area of each negative contour
  negative_areas = [cv2.contourArea(contour) for contour in negative_contours]

  # Calc the score
  total_area = sum(positive_areas) + sum(negative_areas)
  positive_area_ratio = sum(positive_areas) / total_area
  positive_area_ratio = float(round(positive_area_ratio * 100.0))

  # Update the segmented image
  channel_brown = cv2.drawContours(np.zeros(original_image.shape), positive_contours, -1, (1, 1, 1), -1).astype(np.uint8)
  channel_blue = cv2.drawContours(np.zeros(original_image.shape), negative_contours, -1, (1, 1, 1), -1).astype(np.uint8)

  # Get the segmentation
  brown_seg = cv2.multiply(original_image, channel_brown)
  blue_seg = cv2.multiply(original_image, channel_blue)

  # Plot the results
  comparison_plot(original_image, brown_seg, blue_seg,
                title="Extração de Cor | Esperado: {} | Obtido: {}".format(score, positive_area_ratio), images_title=("Imagem Original", "Células Positivas", "Células Negativas"))
# ----------------------------------------------------------------------------------------------------------------------

* Trecho de código que define a interação com o plot (Jupyter Slider).

In [7]:
import os
from ipywidgets import interact, widgets
from PIL import Image
import matplotlib.pyplot as plt

!pip install openpyxl

# Especifique o caminho completo da pasta descompactada
folder_path = '/content/Validação'  # Substitua pelo caminho correto

# Função para listar os arquivos .tiff na pasta descompactada
def list_tiff_files(folder_path):
    tiff_files = [f for f in os.listdir(folder_path) if f.endswith(".tiff")]
    return sorted(tiff_files)

# Obtenha a lista de todos os arquivos .tiff na pasta descompactada
all_tiff_files = list_tiff_files(folder_path)

# Extraia o nome-paciente do nome do arquivo .tiff
def extract_patient_name(filename):
    parts = filename.split("_")
    if len(parts) >= 3:
        return parts[1]
    return ""

# Crie uma lista de pacientes únicos
patients = sorted(list(set(extract_patient_name(filename) for filename in all_tiff_files)))

# Defina o widget suspenso para a escolha do paciente
patient_widget = widgets.Dropdown(
    options=patients,
    value=None,
    description='Paciente:'
)

# Função para listar os arquivos .tiff para um paciente selecionado
def update_files(change):
    patient = change.new
    if patient is not None:
        # Filtra os arquivos com base no paciente selecionado
        patient_files = [filename for filename in all_tiff_files if extract_patient_name(filename) == patient]
        file_widget.options = sorted(patient_files)
        file_widget.value = None  # Limpa a seleção de arquivo ao mudar de paciente

# Defina o widget suspenso para a escolha do arquivo
file_widget = widgets.Dropdown(
    options=[],
    value=None,
    description='Arquivo:'
)

# Observa a mudança de valor no widget de paciente e chama a função de atualização
patient_widget.observe(update_files, 'value')

# Função para mostrar a imagem com base nos valores de Brown e Blue e exibir o resultado da contagem
import numpy as np

# Função para mostrar a imagem com base nos valores de Brown e Blue e exibir o resultado da contagem
def show_image(patient, filename, brown_1_value, brown_2_value, blue_value):
    if patient is not None and filename is not None:
        image_path = os.path.join(folder_path, filename)
        image = Image.open(image_path)

        # Converta a imagem TIFF em uma matriz NumPy
        image_array = np.array(image)

        # Faça o processamento da imagem com os valores de brown_1_value, brown_2_value e blue_value
        # Aqui você pode aplicar os ajustes de cores desejados usando os valores dos sliders

        # Crie uma máscara para ajustar as cores, por exemplo:
        mask_brown = (image_array[:, :, 0] > brown_1_value) & (image_array[:, :, 0] < brown_2_value)
        image_array[mask_brown, 0] = blue_value  # Ajuste a componente vermelha para azul

        # Realize a contagem desejada (substitua isso pelo seu código de contagem)
        count_result = np.sum(mask_brown)  # Por exemplo, contando pixels com a máscara

        # Exiba a imagem
        plt.imshow(image_array)
        plt.axis('off')
        plt.show()

        # Exiba o resultado da contagem abaixo da imagem
        count_widget.value = count_result

        target_image_path = os.path.join(folder_path, filename)

        # Pass the image path and Ki-67 score to the get_dir_ki67_scores function
        ki67_score = get_image_ki67_score(image_array, exp=4, show=False)

        # Display the result below the image
        count_widget.value = ki67_score

        # Display the Ki-67 score in the console
        print(f"Ki-67 Score for {filename}: {ki67_score}%")

# Widget para exibir o resultado da contagem
count_widget = widgets.IntText(value=0, description='Contagem:')

# Interaja com a função show_image
_ = interact(show_image,
             patient=patient_widget,
             filename=file_widget,
             brown_1_value=widgets.IntSlider(min=0, max=255, step=1, value=195, description="Brown 1"),
             brown_2_value=widgets.IntSlider(min=0, max=255, step=1, value=195, description="Brown 2"),
             blue_value=widgets.IntSlider(min=0, max=255, step=1, value=240, description="Blue"))




interactive(children=(Dropdown(description='Paciente:', options=('', '003', '014', '015', '016', '030', '043',…

Ki-67 Score for 23_003_13.tiff: 16.0%
Ki-67 Score for 23_015_18.tiff: 10.0%


Nesta parte o codigo processa todas as imagens de uma vez só dentro da pasta selecionada e baixa os resultados em um xls

In [ ]:
!pip install xlwt

from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd

from IPython.display import display, FileLink

# Especifique o caminho completo da pasta com as imagens
folder_path = '/content/Validação/19-08-23'  # Substitua pelo caminho correto

# Função para listar os arquivos .tiff na pasta
def list_tiff_files(folder_path):
    tiff_files = [f for f in os.listdir(folder_path) if f.endswith(".tiff")]
    return sorted(tiff_files)

# Função para processar uma imagem e retornar o Ki-67 score
def process_image(image_path, brown_1_value, brown_2_value, blue_value):
    image = Image.open(image_path)
    image_array = np.array(image)

    mask_brown = (image_array[:, :, 0] > brown_1_value) & (image_array[:, :, 0] < brown_2_value)
    image_array[mask_brown, 0] = blue_value

    # Realize a contagem desejada (substitua isso pelo seu código de contagem)
    count_result = np.sum(mask_brown)  # Por exemplo, contando pixels com a máscara

    return count_result

# Lista de arquivos TIFF na pasta
all_tiff_files = list_tiff_files(folder_path)

# Criar um DataFrame para armazenar os resultados
results_df = pd.DataFrame(columns=["Arquivo", "Ki-67 Score"])

# Processar e salvar os resultados de todas as imagens no DataFrame
for filename in all_tiff_files:
    image_path = os.path.join(folder_path, filename)

    # Carregar a imagem
    image = Image.open(image_path)
    image_array = np.array(image)

    # Chamar a função get_image_ki67_score para calcular o Ki-67 score
    ki67_score = get_image_ki67_score(image_array)  # Certifique-se de passar os parâmetros corretos

    # Adicionar o resultado ao DataFrame
    results_df = results_df.append({"Arquivo": filename, "Ki-67 Score": ki67_score}, ignore_index=True)

# Salvar o DataFrame em um arquivo XLS
xls_filename = 'resultados.xls'
results_df.to_excel(xls_filename, index=False)

# Exibir o link para baixar o arquivo XLS
xls_link = FileLink(xls_filename)
display(xls_link)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 2.1 MB/s eta 0:00:00


<ipython-input-8-810907cf6f07>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"Arquivo": filename, "Ki-67 Score": ki67_score}, ignore_index=True)
<ipython-input-8-810907cf6f07>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"Arquivo": filename, "Ki-67 Score": ki67_score}, ignore_index=True)
<ipython-input-8-810907cf6f07>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"Arquivo": filename, "Ki-67 Score": ki67_score}, ignore_index=True)
<ipython-input-8-810907cf6f07>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results

KeyboardInterrupt: ignored

In [ ]:
from PIL import Image
import os
import numpy as np
import pandas as pd
from statistics import mode
import concurrent.futures

from IPython.display import display, FileLink

# Especifique o caminho completo da pasta com as imagens
folder_path = '/content/Validação/003'  # Substitua pelo caminho correto

# Função para listar os arquivos .tiff na pasta
def list_tiff_files(folder_path):
    tiff_files = [f for f in os.listdir(folder_path) if f.endswith(".tiff")]
    return sorted(tiff_files)

# Função para calcular a moda de uma lista, com tratamento de empate
def calculate_mode(lst):
    try:
        return mode(lst)
    except StatisticsError:
        return lst[0]

# Função para extrair o resultado esperado do nome do arquivo
def extract_expected_result(filename):
    parts = filename.split("_")
    if len(parts) >= 4:
        return int(parts[-1].split(".")[0])
    return None

# Função para processar uma única imagem
def process_single_image(image_path, blue_value, brown_value):
    image = Image.open(image_path)
    image_array = np.array(image)
    mask_brown = (image_array[:, :, 0] > brown_value)
    image_array[mask_brown, 0] = blue_value
    ki67_score = get_image_ki67_score(image_array)  # Certifique-se de passar os parâmetros corretos
    return ki67_score

# Lista de arquivos TIFF na pasta
all_tiff_files = list_tiff_files(folder_path)

# Criar um DataFrame para armazenar os resultados
results_df = pd.DataFrame(columns=["Arquivo", "Ki-67 Score"])

# Defina os intervalos de valores possíveis para azul e marrom
blue_values = list(range(0, 256))
brown_values = list(range(0, 256))

# Dicionário para registrar os resultados para cada combinação de azul e marrom
results_dict = {}

# Processar e salvar os resultados para todas as combinações de azul e marrom em paralelo
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for blue_value in blue_values:
        for brown_value in brown_values:
            ki67_scores = []
            for filename in all_tiff_files:
                image_path = os.path.join(folder_path, filename)
                future = executor.submit(process_single_image, image_path, blue_value, brown_value)
                ki67_scores.append(future)
            futures.append((blue_value, brown_value, ki67_scores))

# Aguarde a conclusão de todas as tarefas em paralelo
for blue_value, brown_value, ki67_scores in futures:
    ki67_scores = [result.result() for result in ki67_scores]
    expected_result = extract_expected_result(all_tiff_files[0])

    if calculate_mode(ki67_scores) == expected_result:
        results_dict[(blue_value, brown_value)] = ki67_scores

# Encontrar a moda dos valores de azul e marrom que produziram o resultado esperado
if results_dict:
    best_blue, best_brown = calculate_mode(list(results_dict.keys()))
    print(f"Melhor Valor de Azul: {best_blue}")
    print(f"Melhor Valor de Marrom: {best_brown}")

# Usar os melhores valores de azul e marrom para processar todas as imagens
for filename in all_tiff_files:
    image_path = os.path.join(folder_path, filename)

    # Carregar a imagem
    image = Image.open(image_path)
    image_array = np.array(image)

    # Processar a imagem com os melhores valores de azul e marrom
    mask_brown = (image_array[:, :, 0] > best_brown)
    image_array[mask_brown, 0] = best_blue

    # Calcular o Ki-67 score
    ki67_score = get_image_ki67_score(image_array)  # Certifique-se de passar os parâmetros corretos

    # Adicionar o resultado ao DataFrame
    results_df = results_df.append({"Arquivo": filename, "Ki-67 Score": ki67_score}, ignore_index=True)

# Salvar o DataFrame em um arquivo XLS
xls_filename = 'resultados.xls'
results_df.to_excel(xls_filename, index=False)

# Exibir o link para baixar o arquivo XLS
xls_link = FileLink(xls_filename)
display(xls_link)


/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: [Errno 9] Bad file descriptor
  warnings.warn(str(msg))
/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Método para separar os pacientes em pastas

In [ ]:
import os
import shutil

# Especifique o caminho da pasta raiz com as imagens
root_folder = '/content/Validação'  # Substitua pelo caminho correto

# Função para extrair o número do paciente do nome do arquivo
def extract_patient_number(filename):
    parts = filename.split("_")
    if len(parts) >= 2:
        return parts[1]
    return None

# Obtém a lista de arquivos na pasta raiz
all_files = os.listdir(root_folder)

# Itera sobre os arquivos e organiza-os em subpastas por paciente
for filename in all_files:
    if filename.endswith(".tiff"):
        patient_number = extract_patient_number(filename)
        if patient_number:
            # Crie uma pasta para o paciente, se ainda não existir
            patient_folder = os.path.join(root_folder, patient_number)
            if not os.path.exists(patient_folder):
                os.makedirs(patient_folder)
            # Mova o arquivo para a pasta do paciente
            source_path = os.path.join(root_folder, filename)
            dest_path = os.path.join(patient_folder, filename)
            shutil.move(source_path, dest_path)

print("Pacientes separados com sucesso!")


Pacientes separados com sucesso!
